In [1]:
import os
from collections import defaultdict

In [2]:
refseq_to_chm_number = {}

with open("../../chm_to_acc_numbers.tsv") as fh:
    for line in fh:
        if line.startswith('#'):
            continue
        line = line.strip().split("\t")
        refseq_to_chm_number[line[6]] = line[0]

In [ ]:
#transcript

## dict with search results

# {read_n:{chr_n: {trans_n: str, chr_n: str, coordinates : [],total_length : int}}}

reads_summary = defaultdict()

## m8 == outftmt6 
## query_seq_id => target_seq_id => chm_number => identity => length => mismatch => gaps => 
## ==> start_query => stop_query => start_target => stop_target => e_value => bit-score
with open("../../human_transcript_pacbio_reads/result_long_reads_human_t2t_match.m8") as fh:
    for line in fh:
        line = line.strip().split("\t")
        read = line[0]
        identity = float(line[2])
        chr_n = refseq_to_chm_number[line[1]]
        transcript = line[0].split('_')[4]
        start = int(line[-4])
        stop = int(line[-3])
        start_read = int(line[-6])
        stop_read = int(line[-5])
        if start > stop:
#             print('Stop < start, changing...')
#             print(start, stop)
            start, stop = stop, start
#             print('Change to: ', start, stop)
        if start_read > stop_read:
            start_read, stop_read = stop_read, start_read
        if ((stop - start) < 200) or (identity < 0.8):
            continue
        if not read in reads_summary.keys():
            reads_summary[read] = {}
        if not chr_n in reads_summary[read].keys():
#             print('Adding initial coords to chr', chr_n, start, stop)
            reads_summary[read][chr_n] = {
                'transcript_n': transcript,
                'chr_n': chr_n,
                'coordinates_genome': [f"{start}:{stop}"],
                'coordinates_read': [f"{start_read}:{stop_read}"],
                'total_length_genome': stop - start,
                'total_length_read' : stop_read - start_read
            }
        else:
#             nested_longer = None
#             nested_longer_index = None
            not_nested = None
            #difference = None
            for i, coord in enumerate(reads_summary[read][chr_n]['coordinates_genome']):
                start_in = int(coord.split(':')[0])
                stop_in = int(coord.split(':')[1])
#                 print(f'Coords check: {start_in}:{stop_in} to {start}:{stop}')
#                 if start < start_in and stop > stop_in:
#                     difference = (stop - start) - (stop_in - start_in)
#                     nested_longer = True
#                     nested_longer_index = i
#                 elif start < start_in and start_in < stop < stop_in:
#                     if (stop - start) > (stop_in - start_in):
#                         difference = (stop - start) - (stop_in - start_in)
#                         nested_longer = True
#                         nested_longer_index = i
#                 elif start_in < start < stop and stop > stop_in:
#                     if (stop - start) > (stop_in - start_in):
#                         difference = (stop - start) - (stop_in - start_in)
#                         nested_longer = True
#                         nested_longer_index = i
                if (start < start_in) and (stop < start_in):
#                     print('Not Nested 1! Continue check...')
                    not_nested = True
                elif (start > stop_in) and (stop > stop_in):
                    not_nested = True
#                     print('Not nested 2! Continue check...')
                else:
#                     print('Nested! Breaking...')
                    not_nested = False
                    break
#             if nested_longer:
#                 reads_summary[read][chr_n]['coordinates'][nested_longer_index] = f'{start}:{stop}'
#                 reads_summary[read][chr_n]['total_length'] += difference
            if not_nested:
                reads_summary[read][chr_n]['coordinates_genome'].append(f'{start}:{stop}')
                reads_summary[read][chr_n]['total_length_genome'] += stop - start
                if f'{start_read}:{stop_read}' in reads_summary[read][chr_n]['coordinates_read']:
                    continue
                else:
                    not_nested_read = None
                    for i, coord in enumerate(reads_summary[read][chr_n]['coordinates_read']):
                        start_in = int(coord.split(':')[0])
                        stop_in = int(coord.split(':')[1])
                        if (start_read < start_in) and (stop_read < start_in):
                            not_nested_read = True
                        elif (start_read > stop_in) and (stop_read > stop_in):
                            not_nested_read = True
                        else:
                            not_nested_read = False
                            break
                    if not_nested_read:
                        reads_summary[read][chr_n]['coordinates_read'].append(f'{start_read}:{stop_read}')
                        reads_summary[read][chr_n]['total_length_read'] += stop_read - start_read

In [ ]:
import json
with open('../data/reads_match_coords.json', 'w', encoding='utf-8') as f:
    json.dump(reads_summary, f, ensure_ascii=False, indent=2)

In [ ]:
reads_summary_without_coords = {}

for read, chr_info in reads_summary.items():
    print(read)
    for chr_n, info in chr_info.items():
        read_key = f"{read}_chr_{chr_n}"
        print(read_key)
        reads_summary_without_coords[read_key] = [chr_n, info['transcript_n'], str(info['total_length_genome']), str(info['total_length_read'])]

In [ ]:
with open('../data/reads_match_summary.tsv', 'w') as fw:
    fw.write('#read\tchromosome\ttranscript\ttotal_length_genome\ttotal_length_read\n')
    for read, summary in reads_summary_without_coords.items():
        line_to_write = "\t".join(summary)
        fw.write(f"{read}\t{line_to_write}\n")